In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True
%config Completer.use_jedi = False

In [ ]:
# Loading the Data Set
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
K = len(np.unique(y_train)) # Number of Classes

Ntr = x_train.shape[0]
Nte = x_test.shape[0]
Din = 3072 # CIFAR10 # 32x32x3 = height x width x channel

# Normalize pixel values: Image data preprocessing
x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = tf.reduce_mean(x_train, axis=0) # axis=0: mean of a column; Mean of each pixel
x_train = x_train - mean_image
x_test = x_test - mean_image

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
y_test =  tf.keras.utils.to_categorical(y_test,  num_classes=K)

x_train = tf.reshape(x_train,(Ntr,Din)).astype('float32')
x_test = tf.reshape(x_test,(Nte,Din)).astype('float32')

std=1e-5 # For random samples from N(\mu, \sigma^2), use: sigma * np.random.randn(...) + mu
#w1 = std*np.random.randn(Din, K) # Initializing the weight matrix with random weights
w1 = std*tf.random.normal([Din,k], 0, 1)
b1 = tf.zeros([K]) # Initializing the bias vector

# Rearranging train and test samples: (ra=rearranged)
x_train_ra = tf.concat((np.ones((x_train.shape[0],1)),x_train), axis=1)
x_test_ra  = tf.concat((np.ones((x_test.shape[0],1)),x_test), axis=1)

# Rearranging weight matrix and bias matrix into single matrix
w1 = tf.concat((b1.reshape(1,K), w1), axis=0)

In [ ]:
#------------------------------------------------------------------------------------------

iterations = 300  # Gradient descent interations
lr = 1.4e-2 # Learninig rate
lr_decay= 0.999
reg = 5e-6

loss_history = [] # Vlaues of loss function at each iteration 
test_loss = []
train_acc_history = [] # Training accuracy
val_acc_history = [] # Validation accuracy


m = x_train.shape[0]  # Number of training examples
m2 = x_test_ra.shape[0]

# Running gradient descent number of times speciied in iterations
print("Running gradient descent...")

for t in range(1,iterations+1):    
    # Forward Propagation
    hypothesis = x_train_ra.dot(w1)
    loss = (1/(2*m))*np.sum(( hypothesis - y_train)**2) + (1/(2*m))*reg*np.sum(w1**2) 
    loss_history.append(loss)
    
    # Backward Propagation
    dw1 = (1/m)*(x_train_ra.T.dot(hypothesis - y_train))  + (1/m)*reg*w1 
    w1 = w1 - lr*dw1
    
    
    # Training Accuracy and Validation Accuracy
    train_acc = getAccuracy(hypothesis, y_train)
    train_acc_history.append(train_acc)
    valid_acc = getAccuracy(x_test_ra.dot(w1), y_test)
    val_acc_history.append(valid_acc)
   
    # Test Loss    
    t_loss = (1/(2*m2))*np.sum(( x_test_ra.dot(w1) - y_test)**2) + (1/(2*m2))*reg*np.sum(w1**2) 
    test_loss.append(t_loss)
    
    # Print details for selected iterations
    if (t%30==0) or (t==1):
        print("| Epoch {:03} | Loss {:.4f} | accuracy: {:.4f} | val_loss: {:.4f} | val_accuracy: {:.4f} |"\
             .format(t, loss, train_acc, t_loss, valid_acc))
    
    
    # Decaying learning rate
    lr = lr*lr_decay